In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans

In [2]:
# optimized partitional models
clara_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/clara.csv')
fuzzy_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/fuzzy_c_means.csv')
kmeans_opt = pd.read_csv('../2_models/with_raw_data/wind/results/partitional_opt/csv_labels_raw/kmeans.csv')

# partitionalal models
clara = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/clara.csv')
fuzzy = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/fuzzy_c_means.csv')
kmeans = pd.read_csv('../2_models/with_raw_data/wind/results/partitional/csv_labels_raw/kmeans.csv')

# model based
bgmm = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/bgmm.csv')
dpmm = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/dpmm.csv')
gaussian_mix = pd.read_csv('../2_models/with_raw_data/wind/results/model_based/csv_labels_cut_raw/gaussian_mix.csv')

# optimized model based
bgmm_opt = pd.read_csv('../2_models/with_raw_data/wind/results/model_based_opt/csv_labels_cut_raw/bgmm.csv')
gaussian_mix_opt = pd.read_csv('../2_models/with_raw_data/wind/results/model_based_opt/csv_labels_cut_raw/gaussian_mix.csv')

# optimized hierarchical models
average_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/average_link.csv')
birch_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/birch.csv')
single_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/single_link.csv')
centroid_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/centroid_link.csv')
ward_link_opt = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical_opt/csv_labels_cut_raw/ward_link.csv')

# hierarchical models
average_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/average_link.csv')
birch = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/birch.csv')
single_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/single_link.csv')
centroid_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/centroid_link.csv')
ward_link = pd.read_csv('../2_models/with_raw_data/wind/results/hierarchical/csv_labels_cut_raw/ward_link.csv')

# density based models
dbscan = pd.read_csv('../2_models/with_raw_data/wind/results/density_based/csv_labels_cut_raw/DBSCAN.csv')
optics = pd.read_csv('../2_models/with_raw_data/wind/results/density_based/csv_labels_cut_raw/OPTICS.csv')

In [3]:
from scipy.spatial.distance import pdist, cdist

def dunn_index(df):
    points = df[['0', '1', '2']].values
    labels = df['cluster_label'].values
    unique_labels = np.unique(labels)
    
    max_intra_cluster_distances = []
    min_inter_cluster_distance = float('inf')

    for label in unique_labels:
        cluster_points = points[labels == label]
        if len(cluster_points) > 1:
            intra_distances = pdist(cluster_points)
            max_intra_cluster_distances.append(np.max(intra_distances))
        else:
            max_intra_cluster_distances.append(0)

    for i, label_i in enumerate(unique_labels):
        cluster_points_i = points[labels == label_i]
        for j, label_j in enumerate(unique_labels):
            if label_i < label_j:
                cluster_points_j = points[labels == label_j]
                inter_distances = cdist(cluster_points_i, cluster_points_j)
                min_inter_cluster_distance = min(min_inter_cluster_distance, np.min(inter_distances))

    if len(max_intra_cluster_distances) == 0 or min_inter_cluster_distance == float('inf'):
        return 0

    return min_inter_cluster_distance / max(max_intra_cluster_distances)

In [4]:
# def xie_beni_index(df):
#     # Compute the total scatter
#     total_scatter = np.sum(pdist(df[['0', '1', '2']]) ** 2) / (2 * len(df))
    
#     # Compute the intra-cluster scatter
#     cluster_labels = df['cluster_label'].unique()
#     intra_cluster_scatter = 0
#     for label in cluster_labels:
#         cluster_points = df[df['cluster_label'] == label][['0', '1', '2']]
#         cluster_center = np.mean(cluster_points, axis=0)
#         intra_cluster_scatter += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1) ** 2)
#     intra_cluster_scatter /= len(df)
#     return intra_cluster_scatter / total_scatter

In [5]:
# Define a function to compute evaluation metrics
def compute_metrics(df):
    metrics = {}
    
    if df['cluster_label'].nunique() > 1:
        metrics['Silhouette Score'] = silhouette_score(df[['0', '1', '2']], df['cluster_label'])
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(df[['0', '1', '2']], df['cluster_label'])
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(df[['0', '1', '2']], df['cluster_label'])
        metrics['Dunn Index'] = dunn_index(df)
        # metrics['Xie-Beni Index'] = xie_beni_index(df)
    else:
        metrics['Silhouette Score'] = 0
        metrics['Davies-Bouldin Index'] = 0
        metrics['Calinski-Harabasz Index'] = 0
        metrics['Dunn Index'] = 0
        # metrics['Xie-Beni Index'] = 0

    return metrics


In [6]:
dataframes = {
    'clara_opt': clara_opt, 'fuzzy_opt': fuzzy_opt, 'kmeans_opt': kmeans_opt,
    'clara': clara, 'fuzzy': fuzzy, 'kmeans': kmeans,
    'bgmm': bgmm, 'gaussian_mix': gaussian_mix,
    'average_link_opt': average_link_opt, 'birch_opt': birch_opt,
    'single_link_opt': single_link_opt, 'centroid_link_opt': centroid_link_opt,
    'ward_link_opt': ward_link_opt,
    'average_link': average_link, 'birch': birch, 'single_link': single_link,
    'centroid_link': centroid_link, 'ward_link': ward_link,
    'dbscan': dbscan, 'optics': optics,
    'bgmm_optimized': bgmm_opt, 'gaussian_mix_opt' : gaussian_mix_opt
 }

In [7]:
# Iterate over each dataframe and print the name of the ones without a 'cluster_label' column
for model_name, df in dataframes.items():
    if 'cluster_label' not in df.columns:
        print(f"The dataframe '{model_name}' does not have a column named 'cluster_label'.")
        print(df.columns)

The dataframe 'fuzzy_opt' does not have a column named 'cluster_label'.
Index(['0', '1', '2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans_opt' does not have a column named 'cluster_label'.
Index(['0', '1', '2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'clara' does not have a column named 'cluster_label'.
Index(['0', '1', '2', 'cluster'], dtype='object')
The dataframe 'fuzzy' does not have a column named 'cluster_label'.
Index(['0', '1', '2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans' does not have a column named 'cluster_label'.
Index(['0', '1', '2', 'cluster', 'asset_id'], dtype='object')


In [8]:
# Loop through each dataframe and rename the 'cluster' column to 'cluster_label'
for model_name, df in dataframes.items():
    if 'cluster' in df.columns:
        df.rename(columns={'cluster': 'cluster_label'}, inplace=True)

In [9]:
# Initialize an empty list to store the results
results = []

# Compute and store the evaluation metrics for each dataframe
for model_name, df in dataframes.items():
    print(f'Evaluating {model_name}')
    metrics = compute_metrics(df)
    results.append({'Model': model_name, **metrics})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

Evaluating clara_opt


Evaluating fuzzy_opt
Evaluating kmeans_opt
Evaluating clara
Evaluating fuzzy
Evaluating kmeans
Evaluating bgmm
Evaluating gaussian_mix
Evaluating average_link_opt
Evaluating birch_opt
Evaluating single_link_opt
Evaluating centroid_link_opt
Evaluating ward_link_opt
Evaluating average_link
Evaluating birch
Evaluating single_link
Evaluating centroid_link
Evaluating ward_link
Evaluating dbscan
Evaluating optics
Evaluating bgmm_optimized
Evaluating gaussian_mix_opt


In [10]:
results_df

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index
0,clara_opt,0.314599,1.298319,19712.597423,0.000539
1,fuzzy_opt,0.055928,1.633612,11272.629778,0.000432
2,kmeans_opt,0.318602,0.981811,26172.505763,0.000738
3,clara,0.209747,1.392954,16357.570030,0.000555
4,fuzzy,0.232067,1.296084,19851.216730,0.000748
5,kmeans,0.266552,1.120441,22117.690485,0.000590
6,bgmm,0.196860,1.781506,13523.069114,0.000542
7,gaussian_mix,0.219374,1.689784,14991.061634,0.000542
8,average_link_opt,0.293662,0.993784,298.568858,0.019016
9,birch_opt,0.231587,1.431397,14200.554029,0.000679


In [11]:
results_df.to_csv('results/raw_wind.csv', index=False)

In [12]:
results_df = pd.read_csv('results/raw_wind.csv')

In [13]:
# Convert columns to numeric data types
normalized_df = results_df.copy()
for col in normalized_df.columns[1:]:
    normalized_df[col] = pd.to_numeric(normalized_df[col], errors='coerce')

# Drop rows with missing or non-numeric values
normalized_df = normalized_df.dropna()

# Define ideal and anti-ideal solutions for each metric
ideal_solution = normalized_df.copy()
anti_ideal_solution = normalized_df.copy()

# Define metrics where higher values are better
higher_is_better = ['Silhouette Score', 'Dunn Index', 'Calinski-Harabasz Index']

# Define metrics where lower values are better
lower_is_better = ['Davies-Bouldin Index']

# Set ideal and anti-ideal solutions for metrics where higher values are better
for metric in higher_is_better:
    ideal_solution[metric] = normalized_df[metric].max()
    anti_ideal_solution[metric] = normalized_df[metric].min()

# Set ideal and anti-ideal solutions for metrics where lower values are better
for metric in lower_is_better:
    ideal_solution[metric] = normalized_df[metric].min()
    anti_ideal_solution[metric] = normalized_df[metric].max()

# Calculate the distance from each alternative to the ideal and anti-ideal solutions
dist_to_ideal = ((normalized_df.iloc[:,1:] - ideal_solution) ** 2).sum(axis=1) ** 0.5
dist_to_anti_ideal = ((normalized_df.iloc[:,1:] - anti_ideal_solution) ** 2).sum(axis=1) ** 0.5

# Calculate the relative closeness to the ideal solution
normalized_df['Closeness'] = dist_to_anti_ideal / (dist_to_ideal + dist_to_anti_ideal)

# Sort the models based on their relative closeness to the ideal solution
ranked_models = normalized_df.sort_values(by='Closeness', ascending=False)

# Print the ranked models
ranked_models[['Model', 'Closeness']]

,Model,Closeness
2,kmeans_opt,0.999969
14,birch,0.888953
5,kmeans,0.845010
4,fuzzy,0.758378
0,clara_opt,0.753079
17,ward_link,0.750174
12,ward_link_opt,0.733579
3,clara,0.624838
7,gaussian_mix,0.572605
9,birch_opt,0.542389


3 best models: k-means, fuzzy c means, gaussian mix

(not including kmeans_opt since the best model is kmeans)

In [15]:
results_df.iloc[[2,14,5,4]]

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index
2,kmeans_opt,0.318602,0.981811,26172.505763,0.000738
14,birch,0.300501,1.095681,23267.305688,0.001137
5,kmeans,0.266552,1.120441,22117.690485,0.000590
4,fuzzy,0.232067,1.296084,19851.216730,0.000748
